## Data Cleaning for New York

In [1]:
import geopandas as gpd
import maup

maup.progress.enabled = True

### Import the necessary files from the redistricting hub:

- **ny_pl2020_vtd**: https://redistrictingdatahub.org/dataset/new-york-vtd-pl-94171-2020/
- **ny_vest_18**: https://redistrictingdatahub.org/dataset/vest-2018-new-york-precinct-and-election-results/
- **ny_pl2020_cd**: https://redistrictingdatahub.org/dataset/new-york-congressional-district-pl-94171-2020/
- **ny_cong_adopted_2022**: https://redistrictingdatahub.org/dataset/2022-new-york-congressional-districts-plan/

In [2]:
population_df = gpd.read_file("./ny_pl2020_vtd/ny_pl2020_vtd.shp")
election_df = gpd.read_file("./ny_vest_18/ny_vest_18.shp")
cong_df = gpd.read_file("./ny_cong_adopted_2022/CON22_June_03_2022.shp")

#### Let's clean up the column names from ```election_df```, and remove the ones we don't need

In [3]:
election_df.rename(columns={
    "G18USSDGIL": "G18SEND",
    "G18USSRFAR": "G18SENR",
    "G18GOVDCUO": "G18GOVD",
    "G18GOVRMOL": "G18GOVR",
    "G18COMDDIN": "G18COMD",
    "G18COMRTRI": "G18COMR",
    "G18ATGDJAM": "G18ATGD",
    "G18ATGRWOF": "G18ATGR"
}, inplace=True)

election_df.drop(columns=[
    "G18USSOWRI", "G18GOVLSHA", "G18GOVGHAW", "G18GOVSMIN",
    "G18GOVOWRI", "G18COMLGAL", "G18COMGDUN", "G18COMOWRI",
    "G18ATGLGAR", "G18ATGGSUS", "G18ATGOSLI", "G18ATGOWRI"],
                inplace=True)

#### Check to make sure we only have the election columns we need

In [4]:
election_df.columns

Index(['STATEFP', 'COUNTYFP', 'COUNTY', 'PRECINCT', 'G18SEND', 'G18SENR',
       'G18GOVD', 'G18GOVR', 'G18COMD', 'G18COMR', 'G18ATGD', 'G18ATGR',
       'geometry'],
      dtype='object')

#### Find the column from ```cong_df``` that gives us the unique district identifier

In [5]:
cong_df.head()

,OBJECTID,Shape_Leng,Shape_Area,DISTRICT,geometry
0,1,398240.168961,4.747143e+09,1,"POLYGON ((742169.526 4532898.369, 736334.250 4..."
1,2,207672.858852,1.482939e+09,2,"POLYGON ((693595.108 4531758.179, 693510.421 4..."
2,3,146813.371952,6.453502e+08,3,"POLYGON ((632044.962 4535980.849, 632198.219 4..."
3,4,100444.291013,4.891851e+08,4,"POLYGON ((622128.271 4510545.013, 622090.862 4..."
4,5,113036.578196,2.913121e+08,5,"POLYGON ((603301.333 4509330.502, 603368.523 4..."


##### *We'll save column DISTRICT for later*

In [6]:
district_col_name = "DISTRICT"

#### Next we need to align the precincts from the 2018 data with the 2020 census data, but first we must make sure the CRS values match

In [7]:
print(population_df.crs)
print(cong_df.crs)
print(election_df.crs)

EPSG:4269
EPSG:26918
EPSG:3857


##### Looks like the CRS values don't match. This will cause issues while calling ```maup.assign()```, so let's update them to match `population_df`

In [8]:
election_df.to_crs(population_df.crs, inplace=True)
cong_df.to_crs(population_df.crs, inplace=True)

print(election_df.crs)
print(cong_df.crs)

EPSG:4269
EPSG:4269


#### Great! Now lets go ahead and get our mappings between the census and election data

In [9]:
vtds_to_precincts_assignment = maup.assign(election_df.geometry, population_df.geometry)

100%|██████████| 14191/14191 [00:12<00:00, 1179.83it/s]
/Users/rishab/Library/Python/3.9/lib/python/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/Users/rishab/Library/Python/3.9/lib/python/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/Users/rishab/Library/Python/3.9/lib/python/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(source

##### A lot of these columns names don't make sense, but we'll copy the population data columns from `population_df` into `election_df` for now

In [10]:
pop_column_names = ['P0020001', 'P0020002', 'P0020005', 'P0020006',
                    'P0020007', 'P0020008', 'P0020009', 'P0020010']

vap_column_names = ['P0040001', 'P0040002', 'P0040005', 'P0040006',
                    'P0040007', 'P0040008', 'P0040009', 'P0040010']

In [11]:
election_df[pop_column_names] = population_df[pop_column_names].groupby(vtds_to_precincts_assignment).sum()

election_df[pop_column_names].head()

,P0020001,P0020002,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010
0,813.0,479.0,251.0,43.0,3.0,1.0,0.0,8.0
1,2798.0,338.0,2241.0,63.0,12.0,11.0,1.0,5.0
2,1395.0,130.0,1106.0,33.0,7.0,22.0,0.0,7.0
3,3696.0,494.0,2933.0,84.0,17.0,5.0,1.0,10.0
4,1116.0,23.0,1056.0,4.0,0.0,1.0,0.0,0.0


#### Time to check to see if we lost any of the population in the merge

In [12]:
print('population_df:')
print(population_df[pop_column_names].sum())

print('election_df:')
print(election_df[pop_column_names].sum())

population_df:
P0020001    20201249
P0020002     3948032
P0020005    10598907
P0020006     2759022
P0020007       54908
P0020008     1916329
P0020009        6097
P0020010      197107
dtype: int64
election_df:
P0020001    20201249.0
P0020002     3948032.0
P0020005    10598907.0
P0020006     2759022.0
P0020007       54908.0
P0020008     1916329.0
P0020009        6097.0
P0020010      197107.0
dtype: float64


#### And now comes the mapping between 2018 and 2020 data using `maup.prorate`. This will give us population weights that we can use to reassign the district population to the 2020 districts

In [13]:
weights2018 = population_df["P0040001"] / vtds_to_precincts_assignment.map(population_df["P0040001"].groupby(vtds_to_precincts_assignment).sum())
weights2018 = weights2018.fillna(0)

In [14]:
prorated2018 = maup.prorate(vtds_to_precincts_assignment, election_df[pop_column_names], weights2018)

prorated2018.head()

,P0020001,P0020002,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010
0,617.740727,69.604589,458.048948,41.327725,3.987763,22.29522,0.0,2.537667
1,2477.000000,142.000000,2143.000000,11.000000,11.000000,47.00000,0.0,12.000000
2,3393.000000,612.000000,1517.000000,1141.000000,21.000000,18.00000,1.0,14.000000
3,1490.000000,134.000000,1050.000000,154.000000,1.000000,23.00000,0.0,9.000000
4,1497.000000,45.000000,1349.000000,6.000000,4.000000,4.00000,0.0,3.000000


#### Next we'll store the prorated election columns from `election_df` in `population_df`

In [15]:
election_cols = ["G18SEND", "G18SENR", "G18GOVD", "G18GOVR", "G18COMD", "G18COMR", "G18ATGD", "G18ATGR"]

population_df[election_cols] = prorated2018

#### One more check to make sure we didn't lose anyone in the proration step

In [16]:
print(population_df[pop_column_names].sum())
print(election_df[pop_column_names].sum())

P0020001    20201249
P0020002     3948032
P0020005    10598907
P0020006     2759022
P0020007       54908
P0020008     1916329
P0020009        6097
P0020010      197107
dtype: int64
P0020001    20201249.0
P0020002     3948032.0
P0020005    10598907.0
P0020006     2759022.0
P0020007       54908.0
P0020008     1916329.0
P0020009        6097.0
P0020010      197107.0
dtype: float64


#### Perfect! Now that we know we haven't lost anyone, let's make sure `maup.doctor()` runs without any holes in the map

In [17]:
maup.doctor(population_df)

100%|██████████| 14191/14191 [00:07<00:00, 1900.54it/s]
/Users/rishab/Library/Python/3.9/lib/python/site-packages/maup/repair.py:331: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlaps = inters[inters.area > 0].make_valid()


True

#### This step will feel familiar, we'll call `maup.assign()` to map the congressional districts from `cong_df` to the precincts in `population_df`
#### Once we have this assignment, we'll add a new `CD` column to `population_df` with the district that the given precinct falls under

In [18]:
precincts_to_districts_assignment = maup.assign(population_df.geometry, cong_df.geometry)
population_df["CD"] = precincts_to_districts_assignment

for precinct_index in range(len(population_df)):
    population_df.at[precinct_index, "CD"] = int(cong_df.at[population_df.at[precinct_index, "CD"], district_col_name])

100%|██████████| 28/28 [00:00<00:00, 36.49it/s]
/Users/rishab/Library/Python/3.9/lib/python/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/Users/rishab/Library/Python/3.9/lib/python/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/Users/rishab/Library/Python/3.9/lib/python/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targe

#### Almost done! Now it's time to rename those columns from before so that we know what they stand for

In [19]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
                    'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER',
                    'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
                                        'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP'}

population_df.rename(columns=rename_dict, inplace=True)

population_df.columns

Index(['STATEFP20', 'COUNTYFP20', 'VTDST20', 'GEOID20', 'VTDI20', 'NAME20',
       'NAMELSAD20', 'LSAD20', 'MTFCC20', 'FUNCSTAT20',
       ...
       'geometry', 'G18SEND', 'G18SENR', 'G18GOVD', 'G18GOVR', 'G18COMD',
       'G18COMR', 'G18ATGD', 'G18ATGR', 'CD'],
      dtype='object', length=357)

#### Finally, our last step is to save our dataframe into a shapefile we can use for future analysis!

In [20]:
population_df.to_file("./NY/NY.shp")